In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import pickle
import numpy as np

f2 = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/val_data', 'rb')
X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
f2.close()

from keras.preprocessing.sequence import pad_sequences
from util import preprocess

total_X_sequence_train = []
total_X_feature_train = []
# total_X_image_train = []
total_Y_train = []
for i in range(0,10):
    f = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/train_data%d'%i, 'rb')
    X_sequence_train, X_feature_train, X_image_train, Y_train = pickle.load(f)
    X_sequence_train_paded = pad_sequences(X_sequence_train, dtype='float', padding='post', value = np.nan)
    X_sequence_train_trimed = preprocess(X_sequence_train_paded)
    X_feature_train = np.array(X_feature_train)
    Y_train = np.array(Y_train)-1
    total_X_sequence_train.append(X_sequence_train_trimed)
    total_X_feature_train.append(X_feature_train)
    total_Y_train.append(Y_train)
    f.close()

X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
X_sequence_val_trimed = preprocess(X_sequence_val_paded)
X_feature_val = np.array(X_feature_val)
Y_val = np.array(Y_val)-1


In [2]:
def rnn_with_feature(sequence_shape, feature_shape, num_classes):
    sequence_inputs = keras.Input(shape=sequence_shape, name='sequence')
    masking_layer = layers.Masking(mask_value=0)
    x1 = masking_layer(sequence_inputs)
    x1 = layers.Bidirectional(layers.GRU(128, return_sequences=True))(x1)
    x1 = layers.Bidirectional(layers.GRU(64))(x1)
    x1 = layers.Dropout(0.5)(x1)
    x1 = layers.Dense(128, activation="relu")(x1)
    x1 = layers.Dropout(0.5)(x1)
    x1 = layers.Dense(32, activation="relu")(x1)

    feature_inputs = keras.Input(shape=feature_shape, name='feature')
    x3 = layers.Dense(32, activation="relu")(feature_inputs)
    x3 = layers.Dense(16, activation="relu")(x3)

    x = layers.concatenate([x1, x3])
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs=[sequence_inputs, feature_inputs], outputs=outputs)
    return model

In [3]:
model = rnn_with_feature(sequence_shape=(None, 3),feature_shape=(2,), num_classes=11)
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    run_eagerly=False,
)


2022-10-13 12:48:16.378341: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-13 12:48:17.189700: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 507 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:73:00.0, compute capability: 8.6
2022-10-13 12:48:17.190690: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21877 MB memory:  -> device: 1, name: GeForce RTX 3090, pci bus id: 0000:d5:00.0, compute capability: 8.6


In [5]:
X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
for i in range(0,10):
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/bagging/rnn_with_feature%d/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging/rnn_with_feature%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=10, 
        validation_data=(X_val, Y_val), callbacks=callbacks)


2022-10-13 12:49:48.318361: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-13 12:49:48.318422: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-10-13 12:49:48.814893: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-10-13 12:49:48.815104: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


Epoch 1/10


2022-10-13 12:49:49.548935: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-10-13 12:49:49.710522: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8401
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/site-packages/tensorflow/python/../../../../libcublas.so.11: undefined symbol: cublasGetSmCountTarget


  1/860 [..............................] - ETA: 24:38 - loss: 11.0279 - acc: 0.0625

2022-10-13 12:49:51.030249: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-13 12:49:51.030294: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


...

INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature0/model_1/assets


Epoch 2/10


2022-10-13 12:51:39.929541: W tensorflow/core/common_runtime/bfc_allocator.cc:457] Allocator (GPU_0_bfc) ran out of memory trying to allocate 10.83MiB (rounded to 11357440)requested by op cond_41/then/_0/cond/CudnnRNNV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-10-13 12:51:39.929618: I tensorflow/core/common_runtime/bfc_allocator.cc:1004] BFCAllocator dump for GPU_0_bfc
2022-10-13 12:51:39.929640: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (256): 	Total Chunks: 146, Chunks in use: 146. 36.5KiB allocated for chunks. 36.5KiB in use in bin. 3.5KiB client-requested in use in bin.
2022-10-13 12:51:39.929656: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (512): 	Total Chunks: 6, Chunks in use: 6. 3.8KiB allocated for chunks. 3.8KiB in use in bin. 3.8KiB client-requested in use in bin.
2022-10-13

InternalError:    Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 3, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 256, 64, 1, 364, 32, 0] 
	 [[{{node cond_41/then/_0/cond/CudnnRNNV3}}]]
	 [[model/bidirectional_1/forward_gru_1/PartitionedCall]] [Op:__inference_train_function_27288]

Function call stack:
train_function -> train_function -> train_function


2:51:39.933912: I tensorflow/core/common_runtime/bfc_allocator.cc:1060] InUse at 7f29b40cb400 of size 256 next 76
2022-10-13 12:51:39.933920: I tensorflow/core/common_runtime/bfc_allocator.cc:1060] InUse at 7f29b40cb500 of size 256 next 77
2022-10-13 12:51:39.933928: I tensorflow/core/common_runtime/bfc_allocator.cc:1060] InUse at 7f29b40cb600 of size 256 next 79
2022-10-13 12:51:39.933937: I tensorflow/core/common_runtime/bfc_allocator.cc:1060] InUse at 7f29b40cb700 of size 256 next 81
2022-10-13 12:51:39.933945: I tensorflow/core/common_runtime/bfc_allocator.cc:1060] InUse at 7f29b40cb800 of size 256 next 83
2022-10-13 12:51:39.933954: I tensorflow/core/common_runtime/bfc_allocator.cc:1060] InUse at 7f29b40cb900 of size 512 next 97
2022-10-13 12:51:39.933963: I tensorflow/core/common_runtime/bfc_allocator.cc:1060] InUse at 7f29b40cbb00 of size 256 next 98
2022-10-13 12:51:39.933971: I tensorflow/core/common_runtime/bfc_allocator.cc:1060] InUse at 7f29b40cbc00 of size 256 next 99
2022